In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


In [2]:
Data = pd.read_csv('Datasets/Clean_data/Locality_Analyst.csv')

In [3]:
class CombineAttributesAdder(BaseEstimator,TransformerMixin):

    def fit(self,X,y=None):
        X = X.copy()
        return self
    
    def transform(self,X):
        X = X.copy()
        X = X.sort_values(['Vùng',"Năm"])
        X['prev_density'] = X.groupby('Vùng')['Mật độ dân số (Người/km2)'].shift(1)
        X['growth_density'] = X.groupby('Vùng')['Mật độ dân số (Người/km2)'].pct_change()

        return X.reset_index(drop=True)
    
class StrEncoding (BaseEstimator,TransformerMixin):
    def __init__(self):
        self.le_region = LabelEncoder()
        self.le_area = LabelEncoder()

    def fit(self,X,y=None):
        X = X.copy()
        self.le_region.fit(X['Vùng'])
        self.le_area.fit(X['Khu vực'])
        return self

    def transform(self,X):
        X = X.copy()
        X['Vùng'] = self.le_region.fit_transform(X['Vùng'])
        X['Khu vực'] = self.le_area.fit_transform(X['Khu vực'])
        return X
    


num_pipeline = Pipeline([
    ('Labelencode',StrEncoding()),
    ('attribs_adder',CombineAttributesAdder())
])
    

In [4]:
class XGBModel:
    def __init__(self,region_name,features):
        self.region = region_name
        self.features = features
        self.XGBmodel = XGBRegressor(
            n_estimators = 500,
            learning_rate = 0.05,
            max_depth = 6,
            subsample = 0.8,
            colsample_bytree = 0.8,
            random_state = 42
        )

    def get_data(self):
        if self.region == 'All': return num_pipeline.fit_transform(Data)
        return num_pipeline.fit_transform(Data[Data['Vùng'] == self.region])
    
    def data_splits(self,data):
        train = data[data['Năm'] <= 2021]
        test = data[data['Năm'] > 2021]

        return train[self.features], train['Tổng dân số'], test[self.features], test['Tổng dân số']
    

    def train_XGBoost_model(self):
        df = self.get_data()
        X_train,y_train, X_test,y_test = self.data_splits(df)
        self.XGBmodel.fit(X_train,y_train)
        y_pred = self.XGBmodel.predict(X_test)

        return pd.DataFrame(np.column_stack([y_pred,y_test]),columns=['y_pred','y_test']).to_csv(f'Result/Test_set/XGBModel/{self.region}_predict_result.csv')

    def CrossValScore(self,scoring):
        X = num_pipeline.fit_transform(Data)
        prepared = X[self.features]
        label = X['Tổng dân số']
        return cross_val_score(self.XGBmodel,prepared,label,scoring=scoring,cv=5)    


In [7]:
Data.columns

Index(['Vùng', 'Năm', 'Diện tích(Km2)', 'Dân số trung bình (Nghìn người)',
       'Mật độ dân số (Người/km2)', 'Tổng dân số', 'Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Tỷ suất sinh thô',
       'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên', 'Tỷ suất nhập cư',
       'Tỷ suất xuất cư', 'Tỷ suất di cư thuần', 'Tỷ số giới tính', 'Khu vực',
       'Tỉ lệ thất nghiệp', 'Tỉ lệ thất nghiệp ở thành thị',
       'Tỉ lệ thất nghiệp ở nông thôn', 'Phần trăm dân số'],
      dtype='object')

In [8]:
features = ['Vùng', 'Năm', 'Diện tích(Km2)', 'Dân số trung bình (Nghìn người)',
       'Mật độ dân số (Người/km2)','Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Tỷ suất sinh thô',
       'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên', 'Tỷ suất nhập cư',
       'Tỷ suất xuất cư', 'Tỷ suất di cư thuần', 'Tỷ số giới tính', 'Khu vực',
       'Tỉ lệ thất nghiệp', 'Tỉ lệ thất nghiệp ở thành thị',
       'Tỉ lệ thất nghiệp ở nông thôn', 'Phần trăm dân số']

In [9]:
PredictModel = XGBModel('Hà Nội',features)
PredictModel.train_XGBoost_model()
PredictModel.CrossValScore('r2')

array([0.94732175, 0.9476873 , 0.99812334, 0.98676983, 0.99443233])